# Time Series Change Detection for Remote Sensing

This notebook demonstrates how to detect changes in a time series of remote sensing imagery (e.g., Sentinel-2) using Change Vector Analysis (CVA) and a deep learning-based Siamese network with `rasterio`, `geopandas`, `numpy`, `torch`, and `keplergl` in Python. This is useful for monitoring land cover changes, deforestation, or urban expansion over time.

## Prerequisites
- Install required libraries: `rasterio`, `geopandas`, `numpy`, `matplotlib`, `torch`, `keplergl`, `scikit-learn` (listed in `requirements.txt`).
- A stack of preprocessed GeoTIFF files representing a time series (e.g., from `21_download_data.ipynb` or `24_advanced_preprocessing.ipynb`).
- A GeoJSON or shapefile defining the area of interest (AOI) (e.g., `aoi.geojson`).
- Replace file paths with your own data.
- GPU recommended for faster training of the Siamese network.

## Learning Objectives
- Load and preprocess a time series of raster data.
- Perform Change Vector Analysis (CVA) for statistical change detection.
- Train a Siamese network for deep learning-based change detection.
- Visualize change maps interactively using Kepler.gl.

In [ ]:
# Import required libraries
import rasterio
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
from rasterio.warp import transform_bounds
from rasterio.mask import mask
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from keplergl import KeplerGl
import glob
import os
from datetime import datetime
from sklearn.metrics import accuracy_score, confusion_matrix

## Step 1: Load Time Series Rasters and AOI

Load a series of preprocessed GeoTIFF files and the AOI vector data.

In [ ]:
# Define file paths
raster_dir = 'remote_sensing_data/time_series/'  # Directory with time series GeoTIFFs
aoi_path = 'aoi.geojson'                        # Replace with your AOI file
raster_files = sorted(glob.glob(os.path.join(raster_dir, '*.tif')))  # Replace with your file pattern

# Load rasters and extract dates
raster_stack = []
dates = []
for file in raster_files:
    with rasterio.open(file) as src:
        raster_data, transform = mask(src, aoi_gdf.geometry, crop=True, nodata=np.nan)
        raster_stack.append(raster_data)  # Shape: (bands, height, width)
        profile = src.profile
        raster_crs = src.crs
    profile.update({
        'height': raster_data.shape[1],
        'width': raster_data.shape[2],
        'transform': transform,
        'nodata': np.nan
    })
    # Extract date from filename (assumes format like 'sentinel_2023_01_01.tif')
    date_str = os.path.basename(file).split('_')[1]  # Adjust based on your naming convention
    dates.append(datetime.strptime(date_str, '%Y_%m_%d'))

# Convert to numpy array
raster_stack = np.stack(raster_stack, axis=0)  # Shape: (time, bands, height, width)

# Load AOI
aoi_gdf = gpd.read_file(aoi_path)
if aoi_gdf.crs != raster_crs:
    aoi_gdf = aoi_gdf.to_crs(raster_crs)

# Print basic information
print(f'Time series shape: {raster_stack.shape}')
print(f'Dates: {dates}')
print(f'Raster CRS: {raster_crs}')
print(f'AOI CRS: {aoi_gdf.crs}')

## Step 2: Change Vector Analysis (CVA)

Perform CVA to detect changes between consecutive time steps using spectral bands.

In [ ]:
# Select bands for CVA (e.g., Red and NIR for Sentinel-2)
red_band_idx = 0  # Adjust based on your band order
nir_band_idx = 3

# Compute CVA magnitude between consecutive time steps
cva_magnitude = []
for t in range(1, len(raster_stack)):
    t1_data = raster_stack[t-1, [red_band_idx, nir_band_idx]]
    t2_data = raster_stack[t, [red_band_idx, nir_band_idx]]
    delta = t2_data - t1_data
    magnitude = np.sqrt(np.sum(delta ** 2, axis=0))
    cva_magnitude.append(magnitude)

# Convert to numpy array
cva_magnitude = np.stack(cva_magnitude, axis=0)  # Shape: (time-1, height, width)

# Visualize CVA magnitude for the first change
plt.figure(figsize=(8, 8))
plt.imshow(cva_magnitude[0], cmap='hot', vmin=np.nanpercentile(cva_magnitude[0], 2), vmax=np.nanpercentile(cva_magnitude[0], 98))
plt.colorbar(label='Change Magnitude')
plt.title(f'CVA Magnitude ({dates[0].strftime("%Y-%m-%d")} to {dates[1].strftime("%Y-%m-%d")})')
plt.xlabel('Column')
plt.ylabel('Row')
plt.show()

# Save CVA magnitude as GeoTIFF
cva_profile = profile.copy()
cva_profile.update({'count': 1, 'dtype': 'float32'})
for t in range(len(cva_magnitude)):
    cva_output_path = f'remote_sensing_data/cva_magnitude_t{t+1}.tif'
    with rasterio.open(cva_output_path, 'w', **cva_profile) as dst:
        dst.write(cva_magnitude[t], 1)
    print(f'CVA magnitude saved to: {cva_output_path}')

## Step 3: Prepare Dataset for Siamese Network

Create a dataset of image patch pairs with change/no-change labels for training a Siamese network.

In [ ]:
# Define custom dataset class for Siamese network
class ChangeDetectionDataset(Dataset):
    def __init__(self, raster_stack, change_labels, patch_size=64):
        self.raster_stack = raster_stack
        self.change_labels = change_labels  # Binary mask: 1=change, 0=no-change
        self.patch_size = patch_size
        self.pairs = []
        self.labels = []

        # Extract patch pairs
        height, width = raster_stack.shape[2], raster_stack.shape[3]
        for t in range(1, len(raster_stack)):
            for i in range(0, height - patch_size + 1, patch_size//2):
                for j in range(0, width - patch_size + 1, patch_size//2):
                    patch_t1 = raster_stack[t-1, :, i:i+patch_size, j:j+patch_size]
                    patch_t2 = raster_stack[t, :, i:i+patch_size, j:j+patch_size]
                    label = change_labels[t-1, i:i+patch_size, j:j+patch_size].mean() > 0.5
                    if not np.any(np.isnan(patch_t1)) and not np.any(np.isnan(patch_t2)):
                        self.pairs.append((patch_t1, patch_t2))
                        self.labels.append(int(label))

    def __len__(self):
        return len(self.pairs)

    def __getitem__(self, idx):
        patch_t1, patch_t2 = self.pairs[idx]
        label = self.labels[idx]
        patch_t1 = torch.from_numpy(patch_t1.astype(np.float32))
        patch_t2 = torch.from_numpy(patch_t2.astype(np.float32))
        return patch_t1, patch_t2, label

# Generate synthetic change labels for demonstration (replace with actual labels)
change_labels = (cva_magnitude > np.nanpercentile(cva_magnitude, 95, axis=(1, 2), keepdims=True)).astype(np.uint8)

# Create dataset
dataset = ChangeDetectionDataset(raster_stack, change_labels, patch_size=64)

# Split dataset into train and validation sets
train_idx, val_idx = train_test_split(range(len(dataset)), test_size=0.2, random_state=42)
train_dataset = torch.utils.data.Subset(dataset, train_idx)
val_dataset = torch.utils.data.Subset(dataset, val_idx)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

print(f'Training samples: {len(train_dataset)}')
print(f'Validation samples: {len(val_dataset)}')

## Step 4: Define and Train Siamese Network

Set up a Siamese network to learn differences between image pairs for change detection.

In [ ]:
# Define Siamese network
class SiameseNetwork(nn.Module):
    def __init__(self, in_channels):
        super(SiameseNetwork, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.fc = nn.Sequential(
            nn.Linear(256 * 8 * 8, 512),
            nn.ReLU(),
            nn.Linear(512, 2)
        )

    def forward_once(self, x):
        x = self.conv(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

    def forward(self, input1, input2):
        output1 = self.forward_once(input1)
        output2 = self.forward_once(input2)
        return torch.softmax(output1 - output2, dim=1)

# Initialize model
model = SiameseNetwork(in_channels=raster_stack.shape[1]).to(device='cuda' if torch.cuda.is_available() else 'cpu')

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 10
train_losses, val_losses = [], []
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

for epoch in range(num_epochs):
    model.train()
    train_loss = 0
    for patch_t1, patch_t2, labels in train_loader:
        patch_t1, patch_t2, labels = patch_t1.to(device), patch_t2.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(patch_t1, patch_t2)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    train_losses.append(train_loss / len(train_loader))

    model.eval()
    val_loss = 0
    with torch.no_grad():
        for patch_t1, patch_t2, labels in val_loader:
            patch_t1, patch_t2, labels = patch_t1.to(device), patch_t2.to(device), labels.to(device)
            outputs = model(patch_t1, patch_t2)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
    val_losses.append(val_loss / len(val_loader))

    print(f'Epoch {epoch+1}/{num_epochs}, Train Loss: {train_losses[-1]:.4f}, Val Loss: {val_losses[-1]:.4f}')

# Save trained model
torch.save(model.state_dict(), 'siamese_change_detection.pth')
print('Trained model saved to: siamese_change_detection.pth')

## Step 5: Predict and Visualize Change Map

Apply the Siamese network to predict changes across the entire time series.

In [ ]:
# Predict changes for the first time step pair
patch_size = 64
height, width = raster_stack.shape[2], raster_stack.shape[3]
change_map = np.zeros((height, width), dtype=np.uint8)

model.eval()
with torch.no_grad():
    for i in range(0, height - patch_size + 1, patch_size//2):
        for j in range(0, width - patch_size + 1, patch_size//2):
            patch_t1 = raster_stack[0, :, i:i+patch_size, j:j+patch_size]
            patch_t2 = raster_stack[1, :, i:i+patch_size, j:j+patch_size]
            if not np.any(np.isnan(patch_t1)) and not np.any(np.isnan(patch_t2)):
                patch_t1 = torch.from_numpy(patch_t1.astype(np.float32)).unsqueeze(0).to(device)
                patch_t2 = torch.from_numpy(patch_t2.astype(np.float32)).unsqueeze(0).to(device)
                output = model(patch_t1, patch_t2)
                pred = torch.argmax(output, dim=1).cpu().numpy()[0]
                change_map[i:i+patch_size, j:j+patch_size] = pred

# Visualize predicted change map
plt.figure(figsize=(8, 8))
plt.imshow(change_map, cmap='Reds')
plt.title(f'Predicted Change Map ({dates[0].strftime("%Y-%m-%d")} to {dates[1].strftime("%Y-%m-%d")})')
plt.xlabel('Column')
plt.ylabel('Row')
plt.colorbar(ticks=[0, 1], label='Change (0=No, 1=Yes)')
plt.show()

# Save predicted change map as GeoTIFF
change_profile = profile.copy()
change_profile.update({'count': 1, 'dtype': 'uint8', 'nodata': None})
change_output_path = 'remote_sensing_data/change_map.tif'
with rasterio.open(change_output_path, 'w', **change_profile) as dst:
    dst.write(change_map, 1)

print(f'Predicted change map saved to: {change_output_path}')

## Step 6: Visualize with Kepler.gl

Create an interactive visualization of the change map using Kepler.gl.

In [ ]:
# Save change map as PNG for Kepler.gl
change_png_path = 'remote_sensing_data/change_map.png'
imageio.imwrite(change_png_path, change_map * 255)

# Initialize Kepler.gl map
with rasterio.open(change_output_path) as src:
    bounds = src.bounds
bounds_latlon = transform_bounds(raster_crs, 'EPSG:4326', *bounds)

map_config = {
    'version': 'v1',
    'config': {
        'mapState': {
            'latitude': (bounds_latlon[1] + bounds_latlon[3]) / 2,
            'longitude': (bounds_latlon[0] + bounds_latlon[2]) / 2,
            'zoom': 10
        },
        'mapStyle': {
            'styleType': 'dark'
        }
    }
}
m = KeplerGl(height=600, config=map_config)

# Add change map as image layer
m.add_data(
    data={
        'type': 'image',
        'url': change_png_path,
        'bounds': [bounds_latlon[0], bounds_latlon[1], bounds_latlon[2], bounds_latlon[3]]
    },
    name='Change Map'
)

# Add AOI as vector layer
temp_geojson = 'temp_aoi.geojson'
aoi_gdf.to_crs('EPSG:4326').to_file(temp_geojson, driver='GeoJSON')
with open(temp_geojson, 'r') as f:
    m.add_data(data=f.read(), name='AOI')

# Configure visualization settings
m.config['config']['visState'] = {
    'layers': [
        {
            'type': 'grid',
            'config': {
                'dataId': 'Change Map',
                'visualChannels': {'colorField': None, 'colorScale': 'quantile'}
            }
        },
        {
            'type': 'geojson',
            'config': {
                'dataId': 'AOI',
                'visualChannels': {'colorField': None, 'color': [255, 0, 0], 'strokeColor': [255, 0, 0]}
            }
        }
    ]
}

# Display map
m

# Save map to HTML
output_map_path = 'change_map_interactive.html'
m.save_to_html(file_name=output_map_path)
print(f'Interactive change map saved to: {output_map_path}')

# Clean up temporary files
os.remove(temp_geojson)

## Next Steps

- Replace `remote_sensing_data/time_series/*.tif` with your own time series GeoTIFFs (e.g., from `21_download_data.ipynb`).
- Update `aoi.geojson` with your area of interest file.
- Replace synthetic change labels with ground-truth data for more accurate Siamese network training.
- Experiment with other change detection methods (e.g., PCA-based or deep learning with U-Net) for comparison.
- Use the change map in visualization notebooks like `23_kepler_gl_demo.ipynb` or `26_time_series_animation.ipynb`.

## Notes
- Ensure all rasters in the time series have the same CRS, resolution, and dimensions (see `24_advanced_preprocessing.ipynb`).
- CVA is sensitive to band selection; adjust indices (e.g., Red, NIR) based on your data.
- Siamese network performance depends on labeled data quality; consider data augmentation for small datasets.
- See `docs/installation.md` for troubleshooting library installation.